## 问题2    
### 根据所给数据和上述模型，对经中路和纬中路上所有交叉口的信号灯进行优化配置，在保证车辆通行的前提下，使得两条主路上的车流平均速度最大。

In [1]:
import os
import sys

# 先导入 TensorFlow
import tensorflow as tf
import torch
print(torch.__version__)

# 检查 TensorFlow 版本
print("TensorFlow version:", tf.__version__)
print(sys.executable)
# 检查是否有可用的 GPU
try:
    tf_gpus = tf.config.list_physical_devices('GPU')
    if tf_gpus:
        print("GPUs available for TensorFlow:")
        for gpu in tf_gpus:
            print(gpu)
    else:
        print("No GPUs available for TensorFlow.")
except Exception as e:
    print("TensorFlow GPU check failed:", e)

# 再导入 PyTorch
try:
    import torch
    print("PyTorch version:", torch.__version__)
    # 检查是否有可用的 GPU
    torch_gpus = torch.cuda.device_count()
    if torch_gpus > 0:
        print(f"{torch_gpus} GPUs available for PyTorch:")
        for idx in range(torch_gpus):
            print(f"GPU {idx}: {torch.cuda.get_device_name(idx)}")
    else:
        print("No GPUs available for PyTorch.")

    # 检查cuda版本
    cuda_version = torch.version.cuda
    if cuda_version:
        print("CUDA version:", cuda_version)
    # 检查cuDNN版本
    cudnn_version = torch.backends.cudnn.version()
    if cudnn_version:
        print("cuDNN version:", cudnn_version)
except Exception as e:
    print("PyTorch GPU check failed:", e)

# 检查是否有可用的 GPU
if not tf_gpus and torch_gpus == 0:
    print("No GPUs available for both TensorFlow and PyTorch.")
    raise RuntimeError("No GPUs available for both TensorFlow and PyTorch.")

os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 设置可见的 GPU 设备
# 运行 TensorFlow 和 PyTorch 的简单测试
try:
    # TensorFlow 测试
    a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
    b = tf.constant([[5.0, 6.0], [7.0, 8.0]])
    c = tf.matmul(a, b)
    print("TensorFlow matrix multiplication result:\n", c.numpy())
    # PyTorch 测试
    a_torch = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
    b_torch = torch.tensor([[5.0, 6.0], [7.0, 8.0]])
    c_torch = torch.matmul(a_torch, b_torch)
    print("PyTorch matrix multiplication result:\n", c_torch.numpy())
except Exception as e:
    print("Error during TensorFlow or PyTorch test:", e)
    raise

# 检查当前gpu型号
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        print(gpu)
else:
    print("No GPU found.")

if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
else:
    print("No GPU found.")

1.10.1+cu113
TensorFlow version: 2.10.0
c:\Users\hhui8\.conda\envs\Mathmatical\python.exe
GPUs available for TensorFlow:
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
PyTorch version: 1.10.1+cu113
1 GPUs available for PyTorch:
GPU 0: NVIDIA GeForce RTX 4060 Laptop GPU
CUDA version: 11.3
cuDNN version: 8200
TensorFlow matrix multiplication result:
 [[19. 22.]
 [43. 50.]]
PyTorch matrix multiplication result:
 [[19. 22.]
 [43. 50.]]
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
NVIDIA GeForce RTX 4060 Laptop GPU


In [2]:
with tf.device('/gpu:0'):
    # 导入相关库
    import os
    import sys
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns

    # 显示中文
    plt.rcParams['font.sans-serif'] = ['SimHei']
    # 设置负号正常显示
    plt.rcParams['axes.unicode_minus'] = False

    files = os.listdir('./问题2支撑材料')
    for file in files:
        file_path = os.path.join('./问题2支撑材料', file)
        if os.path.isfile(file_path):
            os.remove(file_path)

# 在绘图代码前添加以下配置
import matplotlib as mpl
import matplotlib.font_manager as fm

# 查看系统可用字体
font_list = fm.findSystemFonts()
print("系统中的中文字体:")
for font in font_list:
    if '微软雅黑' in font or 'Microsoft YaHei' in font or 'SimHei' in font or '黑体' in font:
        print(font)

# 使用系统中已安装的中文字体
font_path = 'C:/Windows/Fonts/msyh.ttc'  # 微软雅黑字体路径
font_prop = fm.FontProperties(fname=font_path)

# 设置matplotlib参数
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei', 'SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

系统中的中文字体:


## 思路分析：
### 模型目标
- 优化目标：最大化经中路和纬中路上所有路段的车辆平均速度。
- 约束条件：确保每个交叉口的车流量不超过其通行能力，保证车辆顺畅通行。

### 数据准备

- 输入数据：利用第一问的结果，获取每个时段（低峰、平峰、高峰）、每个方向（东-西、西-东、南-北、北-南）的左转、右转和直行车流量。
- 数据处理：

- 若第一问未直接提供转弯比例，可基于统计方法或假设（如历史数据、路网特征）推断。
- 考虑工作日和节假日的流量差异，可分别建模，或通过加权平均得到综合流量。


- 路网参数：

- 经中路长度：1.8公里，纬中路长度：3.5公里。
- 交叉口数量：12个

In [3]:
import dask.dataframe as dd
from dask import delayed
import dask 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 步骤 1：使用 Dask 读取数据
all_data = dd.read_csv('./题目及附件/附件2.csv', encoding='gbk', blocksize='100MB')

# 步骤 2：获取唯一交叉口列表，排除“经中路-纬中路”
intersection_list = all_data['交叉口'].unique().compute()
intersection_list = [inter for inter in intersection_list if inter != '经中路-纬中路']

# 步骤 3：定义处理已过滤数据的函数
def preprocess_filtered_data(intersection_data, intersection):
    print(f"{intersection} 数据量: {len(intersection_data)}")
    # 检查缺失值
    missing_values = intersection_data.isnull().sum()
    if missing_values.any():
        print(f"缺失值情况 for {intersection}:\n{missing_values[missing_values > 0]}")
    else:
        print(f"无缺失值 for {intersection}")
    
    # 检查重复值
    duplicate_rows = intersection_data.duplicated().sum()
    if duplicate_rows > 0:
        print(f"重复行数 for {intersection}: {duplicate_rows}")
        intersection_data = intersection_data.drop_duplicates()
    else:
        print(f"无重复行 for {intersection}")
    
    # 转换时间格式
    intersection_data['时间'] = pd.to_datetime(intersection_data['时间'], errors='coerce')
    if intersection_data['时间'].isnull().any():
        print(f"时间格式转换失败 for {intersection}, please check data format")
    else:
        print(f"时间格式转换成功 for {intersection}")
    
    # 检查日期范围
    date_range = intersection_data['时间'].dt.date.unique()
    if len(date_range) > 0:
        print(f"数据包含的日期范围 for {intersection}: {date_range[0]} 至 {date_range[-1]}")
    else:
        print(f"数据中没有有效的日期范围 for {intersection}")
    
    # 处理异常车牌
    abnormal_plates = intersection_data[intersection_data['车牌号'] == '无车牌']
    if not abnormal_plates.empty:
        print(f"异常车牌数据 for {intersection}:\n{abnormal_plates}")
        intersection_data = intersection_data[intersection_data['车牌号'] != '无车牌']
    else:
        print(f"无异常车牌数据 for {intersection}")
    
    # 分类工作日和节假日
    intersection_data['工作日'] = intersection_data['时间'].dt.dayofweek < 5
    intersection_data['节假日'] = intersection_data['时间'].dt.dayofweek >= 5
    special_holidays = pd.to_datetime(['2024-04-04', '2024-05-01', '2024-05-02', '2024-05-03', '2024-05-04', '2024-05-05'])
    special_workdays = pd.to_datetime(['2024-04-07', '2024-04-28'])
    intersection_data['节假日'] = intersection_data['时间'].isin(special_holidays) | intersection_data['节假日']
    intersection_data['工作日'] = intersection_data['时间'].isin(special_workdays) | intersection_data['工作日']
    
    # 检查空缺值
    missing_values = intersection_data.isnull().sum()
    if missing_values.any():
        print(f"缺失值情况 for {intersection}:\n{missing_values[missing_values > 0]}")
    else:
        print(f"无缺失值 for {intersection}")
    
    # 保存处理后的数据
    intersection_data.to_csv(f'./问题2支撑材料/{intersection}_processed.csv', index=False, encoding='utf-8-sig')
    print(f"清洗后的数据已保存到 ./问题2支撑材料/{intersection}_processed.csv for {intersection}")
    
# 步骤 4：定义延迟处理函数
@delayed
def process_intersection(intersection, all_data_pd):
    filtered_data = all_data_pd[all_data_pd['交叉口'] == intersection]
    preprocess_filtered_data(filtered_data, intersection)
    return None

# 步骤 5：创建并计算所有任务
all_data_pd = all_data.compute()
tasks = [process_intersection(intersection, all_data_pd) for intersection in intersection_list]
dask.compute(*tasks)

c:\Users\hhui8\.conda\envs\Mathmatical\lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


环东路-纬中路 数据量: 149502
无缺失值 for 环东路-纬中路
经中路-环南路 数据量: 1153000
经二路-纬中路 数据量: 442483
环西路-纬中路 数据量: 2108707
经五路-纬中路 数据量: 196168
无缺失值 for 经中路-环南路
经三路-纬中路 数据量: 813444
经中路-环北路 数据量: 774937
无缺失值 for 环西路-纬中路
经一路-纬中路 数据量: 758636
无缺失值 for 经二路-纬中路
经四路-纬中路 数据量: 71923
纬中路-景区出入口 数据量: 533047
无缺失值 for 经五路-纬中路
无缺失值 for 经三路-纬中路
无缺失值 for 经四路-纬中路
无缺失值 for 经一路-纬中路
无缺失值 for 经中路-环北路
无缺失值 for 纬中路-景区出入口
无重复行 for 环东路-纬中路经中路-纬一路 数据量: 730770

时间格式转换成功 for 环东路-纬中路
数据包含的日期范围 for 环东路-纬中路: 2024-04-02 至 2024-05-05
无缺失值 for 经中路-纬一路


C:\Users\hhui8\AppData\Local\Temp\ipykernel_38640\1220394744.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intersection_data['时间'] = pd.to_datetime(intersection_data['时间'], errors='coerce')
C:\Users\hhui8\AppData\Local\Temp\ipykernel_38640\1220394744.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intersection_data['时间'] = pd.to_datetime(intersection_data['时间'], errors='coerce')


无重复行 for 经四路-纬中路
无重复行 for 经五路-纬中路
无重复行 for 经二路-纬中路
时间格式转换成功 for 经四路-纬中路


C:\Users\hhui8\AppData\Local\Temp\ipykernel_38640\1220394744.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intersection_data['时间'] = pd.to_datetime(intersection_data['时间'], errors='coerce')


...

C:\Users\hhui8\AppData\Local\Temp\ipykernel_38640\1220394744.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intersection_data['工作日'] = intersection_data['时间'].isin(special_workdays) | intersection_data['工作日']
C:\Users\hhui8\AppData\Local\Temp\ipykernel_38640\1220394744.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intersection_data['节假日'] = intersection_data['时间'].dt.dayofweek >= 5
C:\Users\hhui8\AppData\Local\Temp\ipykernel_38640\1220394744.py:60: SettingWithCopyWarning: 
A value is trying to 

无缺失值 for 环东路-纬中路
无缺失值 for 经五路-纬中路
无缺失值 for 经二路-纬中路
无缺失值 for 经中路-环北路
无缺失值 for 纬中路-景区出入口


C:\Users\hhui8\AppData\Local\Temp\ipykernel_38640\1220394744.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intersection_data['工作日'] = intersection_data['时间'].isin(special_workdays) | intersection_data['工作日']
C:\Users\hhui8\AppData\Local\Temp\ipykernel_38640\1220394744.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intersection_data['工作日'] = intersection_data['时间'].isin(special_workdays) | intersection_data['工作日']
C:\Users\hhui8\AppData\Local\Temp\ipykernel_38640\1220394744.py:56: SettingWithCop

无缺失值 for 经三路-纬中路
无缺失值 for 经中路-纬一路
无缺失值 for 经一路-纬中路
无缺失值 for 经中路-环南路
无缺失值 for 环西路-纬中路
清洗后的数据已保存到 ./问题2支撑材料/经四路-纬中路_processed.csv for 经四路-纬中路
清洗后的数据已保存到 ./问题2支撑材料/环东路-纬中路_processed.csv for 环东路-纬中路
清洗后的数据已保存到 ./问题2支撑材料/经五路-纬中路_processed.csv for 经五路-纬中路
清洗后的数据已保存到 ./问题2支撑材料/经二路-纬中路_processed.csv for 经二路-纬中路
清洗后的数据已保存到 ./问题2支撑材料/纬中路-景区出入口_processed.csv for 纬中路-景区出入口
清洗后的数据已保存到 ./问题2支撑材料/经中路-纬一路_processed.csv for 经中路-纬一路
清洗后的数据已保存到 ./问题2支撑材料/经三路-纬中路_processed.csv for 经三路-纬中路
清洗后的数据已保存到 ./问题2支撑材料/经中路-环北路_processed.csv for 经中路-环北路
清洗后的数据已保存到 ./问题2支撑材料/经一路-纬中路_processed.csv for 经一路-纬中路
清洗后的数据已保存到 ./问题2支撑材料/经中路-环南路_processed.csv for 经中路-环南路
清洗后的数据已保存到 ./问题2支撑材料/环西路-纬中路_processed.csv for 环西路-纬中路


(None, None, None, None, None, None, None, None, None, None, None)

In [4]:

import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# 读取所有处理后的CSV文件并重新绘图
for file in os.listdir('./问题2支撑材料'):
    if file.endswith('_processed.csv'):
        try:
            # 提取交叉口名称
            intersection = file.replace('_processed.csv', '')
            print(f"处理交叉口: {intersection}")
            
            # 读取CSV文件
            file_path = os.path.join('./问题2支撑材料', file)
            df = pd.read_csv(file_path)
            
            # 转换时间格式
            df['时间'] = pd.to_datetime(df['时间'])
            
            # 按天聚合
            daily_counts = df.groupby([df['时间'].dt.date, '工作日']).size().reset_index(name='counts')
            daily_counts.rename(columns={daily_counts.columns[0]: '时间'}, inplace=True)
            daily_counts['时间'] = daily_counts['时间'].astype(str)
            
            # 打印聚合后的数据，确认有内容
            print(f"聚合后数据行数: {len(daily_counts)}")
            print(daily_counts.head())
            
            if not daily_counts.empty:
                # 创建plotly图形
                fig = px.bar(daily_counts, x='时间', y='counts', color='工作日',
                             title=f"{intersection} 日均车辆数量",
                             labels={'时间': '日期', 'counts': '车辆数量', '工作日': '工作日'},
                             color_discrete_sequence=px.colors.qualitative.Set1)
                
                # 设置布局
                fig.update_layout(
                    title={
                        'text': f"{intersection} 日均车辆数量",
                        'font': {'size': 20}
                    },
                    xaxis_title='日期',
                    yaxis_title='车辆数量',
                    legend_title='工作日',
                    font=dict(size=14)
                )
                
                # 设置x轴标签旋转
                fig.update_xaxes(tickangle=45,tickmode='array',
                    tickvals=daily_counts['时间'].tolist(),  # 强制显示所有日期
                    ticktext=daily_counts['时间'].tolist())
                
                # 保存为静态图片
                output_path = os.path.join('./问题2支撑材料', f"{intersection}_daily_counts_plotly.png")
                fig.write_image(output_path, width=1200, height=600, scale=2)
                
                
                print(f"图片已保存: {output_path}")
            else:
                print(f"警告: {intersection} 聚合后数据为空，跳过绘图")
                
        except Exception as e:
            print(f"处理 {file} 时出错: {e}")

处理交叉口: 环东路-纬中路
聚合后数据行数: 25
           时间    工作日  counts
0  2024-04-01   True     710
1  2024-04-02   True   11596
2  2024-04-03   True   14231
3  2024-04-04   True    2711
4  2024-04-06  False     660
图片已保存: ./问题2支撑材料\环东路-纬中路_daily_counts_plotly.png
处理交叉口: 环西路-纬中路
聚合后数据行数: 36
           时间   工作日  counts
0  2024-04-01  True   61504
1  2024-04-02  True   47850
2  2024-04-03  True   62819
3  2024-04-04  True   70916
4  2024-04-05  True   61968
图片已保存: ./问题2支撑材料\环西路-纬中路_daily_counts_plotly.png
处理交叉口: 纬中路-景区出入口
聚合后数据行数: 36
           时间   工作日  counts
0  2024-04-01  True    7727
1  2024-04-02  True    8620
2  2024-04-03  True    9264
3  2024-04-04  True   13523
4  2024-04-05  True   13356
图片已保存: ./问题2支撑材料\纬中路-景区出入口_daily_counts_plotly.png
处理交叉口: 经一路-纬中路
聚合后数据行数: 36
           时间   工作日  counts
0  2024-04-01  True   22680
1  2024-04-02  True   23517
2  2024-04-03  True   27579
3  2024-04-04  True   25442
4  2024-04-05  True   28832
图片已保存: ./问题2支撑材料\经一路-纬中路_daily_counts_plotly.png
处理交叉口: 经三路-纬中路

In [5]:

with tf.device('/GPU:0'):
    # 定义路口列表
    intersections = [
    '环东路-纬中路', '环西路-纬中路', '经二路-纬中路', '经三路-纬中路',
    '经四路-纬中路', '经中路-环北路', '经中路-环南路', '经中路-纬一路',
    '纬中路-景区出入口', '经一路-纬中路', '经五路-纬中路'
]

def process_intersection_hourly_data(intersection_name):
    """
    处理单个路口的每小时车流量数据
    """
    file_path = f'./问题2支撑材料/{intersection_name}_processed.csv' # 问题2支撑材料
    
    if not os.path.exists(file_path): 
        print(f"文件不存在: {file_path}")
        return None, None
    
    try:
        # 读取数据
        df = pd.read_csv(file_path)
        
        # 转换时间格式
        df['时间'] = pd.to_datetime(df['时间'])
        
        # 提取小时
        df['小时'] = df['时间'].dt.hour
        
        # 分别统计工作日和节假日每小时的车流量
        workday_stats = df[df['工作日'] == True].groupby('小时').size().reset_index(name='车流量')
        holiday_stats = df[df['节假日'] == True].groupby('小时').size().reset_index(name='车流量')
        
        # 添加路口名称
        workday_stats['路口'] = intersection_name
        workday_stats['类型'] = '工作日'
        holiday_stats['路口'] = intersection_name
        holiday_stats['类型'] = '节假日'
        
        # 确保24小时都有数据（没有的补0）
        all_hours = pd.DataFrame({'小时': range(24)})
        
        workday_complete = pd.merge(all_hours, workday_stats, on='小时', how='left').fillna(0)
        workday_complete['路口'] = intersection_name
        workday_complete['类型'] = '工作日'
        
        holiday_complete = pd.merge(all_hours, holiday_stats, on='小时', how='left').fillna(0)
        holiday_complete['路口'] = intersection_name
        holiday_complete['类型'] = '节假日'
        
        return workday_complete, holiday_complete
        
    except Exception as e:
        print(f"处理{intersection_name}时出错: {e}")
        return None, None

def create_summary_report():
    """创建汇总报告"""
    all_data = []
    
    for intersection in intersections:
        file_path = f'./问题2支撑材料/{intersection}_processed.csv'
        if not os.path.exists(file_path):
            continue
            
        workday_df, holiday_df = process_intersection_hourly_data(intersection)
        
        if workday_df is not None:
            all_data.append(workday_df)
        if holiday_df is not None:
            all_data.append(holiday_df)
    
    if all_data:
        # 合并所有数据
        final_df = pd.concat(all_data, ignore_index=True)
        
        # 保存汇总数据
        final_df.to_csv('./问题2支撑材料/所有路口每小时车流量统计.csv', index=False, encoding='utf-8-sig')
        
        # 创建透视表
        pivot_df = final_df.pivot_table(
            index='小时', 
            columns=['路口', '类型'], 
            values='车流量', 
            fill_value=0
        )
        
        # 保存透视表
        pivot_df.to_csv('./问题2支撑材料/所有路口每小时车流量透视表.csv', encoding='utf-8-sig')
        
        return final_df, pivot_df
    
    return None, None

# 主程序
if __name__ == "__main__":
    print("开始统计每个路口每小时的车流量数据...")
    
    # 存储所有路口的统计结果
    all_workday_stats = []
    all_holiday_stats = []
    
    # 处理每个路口的数据
    processed_count = 0
    for intersection in intersections:
        print(f"正在处理路口: {intersection}")
        workday_df, holiday_df = process_intersection_hourly_data(intersection)
        
        if workday_df is not None:
            all_workday_stats.append(workday_df)
            all_holiday_stats.append(holiday_df)
            processed_count += 1
            
            # 保存单个路口的统计结果
            workday_df.to_csv(f'./问题2支撑材料/{intersection}_工作日每小时车流量.csv', index=False, encoding='utf-8-sig')
            holiday_df.to_csv(f'./问题2支撑材料/{intersection}_节假日每小时车流量.csv', index=False, encoding='utf-8-sig')
    
    # 创建汇总报告
    final_df, pivot_df = create_summary_report()
    
    print(f"\n=== 统计完成 ===")
    print(f"共处理了 {processed_count} 个路口")
    
    if final_df is not None:
        print(f"\n总数据行数: {len(final_df)}")
        
        # 显示汇总统计
        summary_stats = final_df.groupby(['路口', '类型'])['车流量'].sum().reset_index()
        summary_stats.to_csv('./问题2支撑材料/各路口总车流量统计.csv', index=False, encoding='utf-8-sig')
        
        print("\n各路口总车流量统计:")
        print(summary_stats)
        


print("\n所有统计文件已生成完成！")
print("生成的文件包括:")
print("1. 所有路口每小时车流量统计.csv - 所有路口的完整数据")
print("2. 所有路口每小时车流量透视表.csv - 按路口和类型的透视表")
print("3. 各路口总车流量统计.csv - 各路口总流量汇总")
print("4. 每个路口单独的工作日和节假日每小时车流量文件")


开始统计每个路口每小时的车流量数据...
正在处理路口: 环东路-纬中路
正在处理路口: 环西路-纬中路
正在处理路口: 经二路-纬中路
正在处理路口: 经三路-纬中路
正在处理路口: 经四路-纬中路
正在处理路口: 经中路-环北路
正在处理路口: 经中路-环南路
正在处理路口: 经中路-纬一路
正在处理路口: 纬中路-景区出入口
正在处理路口: 经一路-纬中路
正在处理路口: 经五路-纬中路

=== 统计完成 ===
共处理了 11 个路口

总数据行数: 528

各路口总车流量统计:
           路口   类型        车流量
0     环东路-纬中路  工作日   101011.0
1     环东路-纬中路  节假日    32800.0
2     环西路-纬中路  工作日  1491036.0
3     环西路-纬中路  节假日   549195.0
4   纬中路-景区出入口  工作日   376582.0
5   纬中路-景区出入口  节假日   151376.0
6     经一路-纬中路  工作日   553305.0
7     经一路-纬中路  节假日   198995.0
8     经三路-纬中路  工作日   554856.0
9     经三路-纬中路  节假日   193057.0
10    经中路-环北路  工作日   563439.0
11    经中路-环北路  节假日   194231.0
12    经中路-环南路  工作日   817722.0
13    经中路-环南路  节假日   321810.0
14    经中路-纬一路  工作日   509566.0
15    经中路-纬一路  节假日   214393.0
16    经二路-纬中路  工作日   330835.0
17    经二路-纬中路  节假日   108319.0
18    经五路-纬中路  工作日   138137.0
19    经五路-纬中路  节假日    56502.0
20    经四路-纬中路  工作日    51411.0
21    经四路-纬中路  节假日    19413.0

所有统计文件已生成完成！
生成的文件包括:
1. 所有路口每小时车流量统计.csv - 所有路口的完整数据
2. 所有路

In [6]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

with tf.device('/GPU:0'):
    # 定义路口列表
    intersections = [
        '环东路-纬中路', '环西路-纬中路', '经二路-纬中路', '经三路-纬中路',
        '经四路-纬中路', '经中路-环北路', '经中路-环南路', '经中路-纬一路',
        '纬中路-景区出入口', '经一路-纬中路'
    ]

    def plot_hourly_traffic(intersection):
        """为单个路口绘制每小时车流量折线图"""
        workday_file = f'./问题2支撑材料/{intersection}_工作日每小时车流量.csv'
        holiday_file = f'./问题2支撑材料/{intersection}_节假日每小时车流量.csv'
        
        if not os.path.exists(workday_file) or not os.path.exists(holiday_file):
            print(f"数据文件缺失 for {intersection}")
            return
        
        # 读取数据
        workday_df = pd.read_csv(workday_file)
        holiday_df = pd.read_csv(holiday_file)
        
        # 确保小时列是整数
        workday_df['小时'] = workday_df['小时'].astype(int)
        holiday_df['小时'] = holiday_df['小时'].astype(int)
        
        # 创建图表
        plt.figure(figsize=(12, 6))
        
        # 绘制工作日折线
        plt.plot(workday_df['小时'], workday_df['车流量'], marker='o', label='工作日', color='blue', linewidth=2)
        
        # 绘制节假日折线
        plt.plot(holiday_df['小时'], holiday_df['车流量'], marker='s', label='节假日', color='red', linewidth=2)
        
        # 设置标题和标签
        plt.title(f'{intersection} 每小时车流量统计', fontsize=16)
        plt.xlabel('小时', fontsize=14)
        plt.ylabel('车流量', fontsize=14)
        plt.xticks(range(0, 24), fontsize=12)
        plt.yticks(fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend(fontsize=12)
        
        # 保存图片
        output_path = f'./问题2支撑材料/{intersection}_每小时车流量折线图.png'
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f'图片已保存: {output_path}')

    # 主程序
    if __name__ == "__main__":
        print("开始生成每小时车流量折线图...")
        
        for intersection in intersections:
            print(f"处理路口: {intersection}")
            plot_hourly_traffic(intersection)
        
        print("\n所有路口的折线图已生成完成！")
    print("每个路口的图片保存在: ./问题2支撑材料/{路口}_每小时车流量折线图.png")
    print("例如: 环东路-纬中路_每小时车流量折线图.png")

开始生成每小时车流量折线图...
处理路口: 环东路-纬中路
图片已保存: ./问题2支撑材料/环东路-纬中路_每小时车流量折线图.png
处理路口: 环西路-纬中路
图片已保存: ./问题2支撑材料/环西路-纬中路_每小时车流量折线图.png
处理路口: 经二路-纬中路
图片已保存: ./问题2支撑材料/经二路-纬中路_每小时车流量折线图.png
处理路口: 经三路-纬中路
图片已保存: ./问题2支撑材料/经三路-纬中路_每小时车流量折线图.png
处理路口: 经四路-纬中路
图片已保存: ./问题2支撑材料/经四路-纬中路_每小时车流量折线图.png
处理路口: 经中路-环北路
图片已保存: ./问题2支撑材料/经中路-环北路_每小时车流量折线图.png
处理路口: 经中路-环南路
图片已保存: ./问题2支撑材料/经中路-环南路_每小时车流量折线图.png
处理路口: 经中路-纬一路
图片已保存: ./问题2支撑材料/经中路-纬一路_每小时车流量折线图.png
处理路口: 纬中路-景区出入口
图片已保存: ./问题2支撑材料/纬中路-景区出入口_每小时车流量折线图.png
处理路口: 经一路-纬中路
图片已保存: ./问题2支撑材料/经一路-纬中路_每小时车流量折线图.png

所有路口的折线图已生成完成！
每个路口的图片保存在: ./问题2支撑材料/{路口}_每小时车流量折线图.png
例如: 环东路-纬中路_每小时车流量折线图.png


In [7]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

# 定义路口列表
intersections = [
    '环东路-纬中路', '环西路-纬中路', '经二路-纬中路', '经三路-纬中路',
    '经四路-纬中路', '经中路-环北路', '经中路-环南路', '经中路-纬一路',
    '纬中路-景区出入口', '经一路-纬中路'
]

def plot_hourly_traffic_bar(intersection):
    """为单个路口绘制每小时车流量柱状图"""
    workday_file = f'./问题2支撑材料/{intersection}_工作日每小时车流量.csv'
    holiday_file = f'./问题2支撑材料/{intersection}_节假日每小时车流量.csv'
    
    if not os.path.exists(workday_file) or not os.path.exists(holiday_file):
        print(f"数据文件缺失 for {intersection}")
        return
    
    # 读取数据
    workday_df = pd.read_csv(workday_file)
    holiday_df = pd.read_csv(holiday_file)
    
    # 确保小时列是整数并排序
    workday_df['小时'] = workday_df['小时'].astype(int)
    holiday_df['小时'] = holiday_df['小时'].astype(int)
    workday_df = workday_df.sort_values('小时')
    holiday_df = holiday_df.sort_values('小时')
    
    # 创建图表
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # 设置柱状图宽度和位置
    hours = workday_df['小时']
    bar_width = 0.4
    index = np.arange(len(hours))
    
    # 绘制工作日柱状
    ax.bar(index - bar_width/2, workday_df['车流量'], bar_width, label='工作日', color='blue', alpha=0.8)
    
    # 绘制节假日柱状
    ax.bar(index + bar_width/2, holiday_df['车流量'], bar_width, label='节假日', color='red', alpha=0.8)
    
    # 设置标题和标签
    ax.set_title(f'{intersection} 每小时车流量统计', fontsize=16)
    ax.set_xlabel('小时', fontsize=14)
    ax.set_ylabel('车流量', fontsize=14)
    ax.set_xticks(index)
    ax.set_xticklabels(hours, fontsize=12, rotation=0)
    ax.tick_params(axis='y', labelsize=12)
    ax.grid(True, axis='y', linestyle='--', alpha=0.7)
    ax.legend(fontsize=12)
    
    # 保存图片
    output_path = f'./问题2支撑材料/{intersection}_每小时车流量柱状图.png'
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close(fig)
    
    print(f'图片已保存: {output_path}')

# 主程序
if __name__ == "__main__":
    print("开始生成每小时车流量柱状图...")
    
    for intersection in intersections:
        print(f"处理路口: {intersection}")
        plot_hourly_traffic_bar(intersection)
    
    print("\n所有路口的柱状图已生成完成！")
print("每个路口的图片保存在: ./问题2支撑材料/{路口}_每小时车流量柱状图.png")
print("例如: 环东路-纬中路_每小时车流量柱状图.png")

开始生成每小时车流量柱状图...
处理路口: 环东路-纬中路
图片已保存: ./问题2支撑材料/环东路-纬中路_每小时车流量柱状图.png
处理路口: 环西路-纬中路
图片已保存: ./问题2支撑材料/环西路-纬中路_每小时车流量柱状图.png
处理路口: 经二路-纬中路
图片已保存: ./问题2支撑材料/经二路-纬中路_每小时车流量柱状图.png
处理路口: 经三路-纬中路
图片已保存: ./问题2支撑材料/经三路-纬中路_每小时车流量柱状图.png
处理路口: 经四路-纬中路
图片已保存: ./问题2支撑材料/经四路-纬中路_每小时车流量柱状图.png
处理路口: 经中路-环北路
图片已保存: ./问题2支撑材料/经中路-环北路_每小时车流量柱状图.png
处理路口: 经中路-环南路
图片已保存: ./问题2支撑材料/经中路-环南路_每小时车流量柱状图.png
处理路口: 经中路-纬一路
图片已保存: ./问题2支撑材料/经中路-纬一路_每小时车流量柱状图.png
处理路口: 纬中路-景区出入口
图片已保存: ./问题2支撑材料/纬中路-景区出入口_每小时车流量柱状图.png
处理路口: 经一路-纬中路
图片已保存: ./问题2支撑材料/经一路-纬中路_每小时车流量柱状图.png

所有路口的柱状图已生成完成！
每个路口的图片保存在: ./问题2支撑材料/{路口}_每小时车流量柱状图.png
例如: 环东路-纬中路_每小时车流量柱状图.png


In [8]:
with tf.device('/GPU:0'):

    # 定义路口列表（基于历史上下文中的文件）
    intersections = [
        '环东路-纬中路',
        '环西路-纬中路',
        '纬中路-景区出入口',
        '经一路-纬中路',
        '经三路-纬中路',
        '经中路-环北路',
        '经中路-环南路',
        '经中路-纬一路',
        '经二路-纬中路',
        '经五路-纬中路',
        '经四路-纬中路'
    ]

    # 输出文件夹
    output_dir = './问题2支撑材料/'

    for intersection in intersections:
        # 读取_processed.csv文件
        file_path = os.path.join(output_dir, f'{intersection}_processed.csv')
        if not os.path.exists(file_path):
            print(f'文件 {file_path} 不存在，跳过 {intersection}')
            continue
        
        df = pd.read_csv(file_path, encoding='utf-8', parse_dates=['时间'])
        
        # 确保“时间”列为小时整数
        df['小时'] = df['时间'].dt.hour
        
        # 只取工作日数据
        df_workday = df[df['工作日']]
        
        # 生成透视表：行是小时，列是方向，值是车流量
        pivot_table_workday = df_workday.pivot_table(index='小时', columns='方向', values='车牌号', aggfunc='count', fill_value=0)
        
        # 重置索引，方便保存和查看
        pivot_table_workday = pivot_table_workday.reset_index()
        
        # 保存为csv
        pivot_table_workday.to_csv(os.path.join(output_dir, f'{intersection}_工作日各方向分时段车流量.csv'), index=False, encoding='gbk')
        
        # 只取节假日数据
        df_holiday = df[~df['工作日']]
        
        # 生成透视表：行是小时，列是方向，值是车流量
        pivot_table_holiday = df_holiday.pivot_table(index='小时', columns='方向', values='车牌号', aggfunc='count', fill_value=0)
        
        # 重置索引，方便保存和查看
        pivot_table_holiday = pivot_table_holiday.reset_index()
        
        # 保存为csv
        pivot_table_holiday.to_csv(os.path.join(output_dir, f'{intersection}_节假日各方向分时段车流量.csv'), index=False, encoding='gbk')
        
        print(f'已为 {intersection} 生成统计文件')

    print('所有路口统计完成')

已为 环东路-纬中路 生成统计文件
已为 环西路-纬中路 生成统计文件
已为 纬中路-景区出入口 生成统计文件
已为 经一路-纬中路 生成统计文件
已为 经三路-纬中路 生成统计文件
已为 经中路-环北路 生成统计文件
已为 经中路-环南路 生成统计文件
已为 经中路-纬一路 生成统计文件
已为 经二路-纬中路 生成统计文件
已为 经五路-纬中路 生成统计文件
已为 经四路-纬中路 生成统计文件
所有路口统计完成


In [9]:
with tf.device('/GPU:0'):
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_score
    from sklearn.preprocessing import StandardScaler
    import numpy as np
    import matplotlib.pyplot as plt
    import os
    import pandas as pd

    # 定义函数来计算肘部法则和轮廓系数
    def plot_elbow_and_silhouette(data, max_k=10, intersection='', day_type='', save_path='.'):
        sse = []
        silhouette_scores = []
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(data)
        
        for k in range(1, max_k + 1):
            kmeans = KMeans(n_clusters=k, random_state=42)
            kmeans.fit(scaled_data)
            sse.append(kmeans.inertia_)
            if k > 1:
                score = silhouette_score(scaled_data, kmeans.labels_)
                silhouette_scores.append(score)
            else:
                silhouette_scores.append(0)  # k=1 无轮廓分数
        
        # 绘制肘部法则图
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.plot(range(1, max_k + 1), sse, marker='o')
        plt.title(f'{intersection} {day_type} 肘部法则')
        plt.xlabel('K 值')
        plt.ylabel('SSE')
        
        # 绘制轮廓系数图
        plt.subplot(1, 2, 2)
        plt.plot(range(1, max_k + 1), silhouette_scores, marker='o')
        plt.title(f'{intersection} {day_type} 轮廓系数')
        plt.xlabel('K 值')
        plt.ylabel('轮廓分数')
        
        plt.savefig(os.path.join(save_path, f'{intersection}_{day_type}_elbow_silhouette.png'))
        plt.close()
        
        # 以肘部法则为主，选择 K（查找 SSE 下降最快的拐点）
        deltas = np.diff(sse)
        k_elbow = np.argmin(deltas) + 2  # +2 因为 diff 减少一个索引
        return k_elbow

    # 定义函数进行 K-means 聚类并绘制结果（类似于第二张图片）
    def plot_kmeans_clusters(data, k, intersection='', day_type='', save_path='.'):
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(data)
        
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(scaled_data)
        
        # 假设数据是时间序列的车流量，x轴为时间或小时
        plt.figure(figsize=(8, 6))
        for cluster in range(k):
            cluster_data = data[labels == cluster]
            plt.scatter(cluster_data.iloc[:, 0], cluster_data.iloc[:, 1], label=f'Cluster {cluster}')
        plt.title(f'{intersection} {day_type} K-means 聚类结果 (K={k})')
        plt.xlabel('特征1 (e.g., 时间)')
        plt.ylabel('特征2 (e.g., 车流量)')
        plt.legend()
        plt.savefig(os.path.join(save_path, f'{intersection}_{day_type}_kmeans_clusters.png'))
        plt.close()
        return labels
        # 主循环：处理每个路口的_processed.csv
    save_path = './问题2支撑材料'
    for file in os.listdir(save_path):
        if file.endswith('_processed.csv'):
            intersection = file.replace('_processed.csv', '')
            df = pd.read_csv(os.path.join(save_path, file))
            df['时间'] = pd.to_datetime(df['时间'])
            df['小时'] = df['时间'].dt.hour
            
            # 分离工作日和节假日
            workday_data = df[df['工作日']]
            holiday_data = df[df['节假日']]
            
            # 创建 pivot tables
            pivot_table_workday = workday_data.pivot_table(index='小时', columns='方向', values='车牌号', aggfunc='count', fill_value=0).reset_index()
            pivot_table_holiday = holiday_data.pivot_table(index='小时', columns='方向', values='车牌号', aggfunc='count', fill_value=0).reset_index()
            
            # 假设数据特征为小时和车流量计数
            workday_features = workday_data.groupby('小时').size().reset_index(name='车流量')[['小时', '车流量']]
            holiday_features = holiday_data.groupby('小时').size().reset_index(name='车流量')[['小时', '车流量']]
            
            labels_workday = None
            labels_holiday = None
            
            if not workday_features.empty:
                k_workday = plot_elbow_and_silhouette(workday_features, max_k=10, intersection=intersection, day_type='工作日', save_path=save_path)
                labels_workday = plot_kmeans_clusters(workday_features, k_workday, intersection=intersection, day_type='工作日', save_path=save_path)
            
            if not holiday_features.empty:
                k_holiday = plot_elbow_and_silhouette(holiday_features, max_k=10, intersection=intersection, day_type='节假日', save_path=save_path)
                labels_holiday = plot_kmeans_clusters(holiday_features, k_holiday, intersection=intersection, day_type='节假日', save_path=save_path)
            
            # 为每一个路口聚类后的数据保存成csv文件
            if not workday_features.empty:
                workday_features['聚类标签'] = labels_workday
                # 添加时段赋值（基于聚类平均车流量排序）
                cluster_means = workday_features.groupby('聚类标签')['车流量'].mean().sort_values()
                time_periods = {label: period for label, period in zip(cluster_means.index, ['低峰', '平峰', '高峰'])}
                workday_features['时段'] = workday_features['聚类标签'].map(time_periods)
                workday_features.to_csv(os.path.join(save_path, f'{intersection}_工作日_clusters.csv'), index=False, encoding='gbk')
            
            if not holiday_features.empty:
                holiday_features['聚类标签'] = labels_holiday
                # 添加时段赋值
                cluster_means = holiday_features.groupby('聚类标签')['车流量'].mean().sort_values()
                time_periods = {label: period for label, period in zip(cluster_means.index, ['低峰', '平峰', '高峰'])}
                holiday_features['时段'] = holiday_features['聚类标签'].map(time_periods)
                holiday_features.to_csv(os.path.join(save_path, f'{intersection}_节假日_clusters.csv'), index=False, encoding='gbk')
            
            # 合并聚类标签和时段到透视表
            pivot_table_workday = pd.merge(
                pivot_table_workday,
                workday_features[['小时', '聚类标签', '时段']],
                on='小时', how='left'
            )
            
            # 检查并获取实际的列名
            print(f"工作日透视表列名: {pivot_table_workday.columns.tolist()}")
            
            # 确保所需的列存在，否则使用实际存在的列
            available_cols = pivot_table_workday.columns.tolist()
            required_cols = ['小时', '1', '2', '3', '4', '聚类标签', '时段']
            
            # 只使用实际存在的列
            cols_to_use = [col for col in required_cols if col in available_cols]
            
            # 如果缺少方向列(1,2,3,4)，尝试使用实际的方向列
            if not all(str(i) in available_cols for i in range(1, 5)):
                # 获取除了'小时','聚类标签','时段'外的列作为方向列
                direction_cols = [col for col in available_cols if col not in ['小时', '聚类标签', '时段']]
                cols_to_use = ['小时'] + direction_cols + ['聚类标签', '时段']
            
            pivot_table_workday = pivot_table_workday[cols_to_use]
            # 保存结果
            pivot_table_workday.to_csv(os.path.join(save_path, f'{intersection}_工作日分方向分时段车流量_带聚类标签.csv'), index=False, encoding='gbk')
            # 显示前几行
            print(pivot_table_workday.head())
            
            # 类似处理节假日
            pivot_table_holiday = pd.merge(
                pivot_table_holiday,
                holiday_features[['小时', '聚类标签', '时段']],
                on='小时', how='left'
            )
            
            # 检查并获取实际的列名
            print(f"节假日透视表列名: {pivot_table_holiday.columns.tolist()}")
            
            # 使用与工作日相同的逻辑处理列
            available_cols = pivot_table_holiday.columns.tolist()
            
            # 只使用实际存在的列
            cols_to_use = [col for col in required_cols if col in available_cols]
            
            # 如果缺少方向列(1,2,3,4)，尝试使用实际的方向列
            if not all(str(i) in available_cols for i in range(1, 5)):
                # 获取除了'小时','聚类标签','时段'外的列作为方向列
                direction_cols = [col for col in available_cols if col not in ['小时', '聚类标签', '时段']]
                cols_to_use = ['小时'] + direction_cols + ['聚类标签', '时段']
            
            pivot_table_holiday = pivot_table_holiday[cols_to_use]
            pivot_table_holiday.to_csv(os.path.join(save_path, f'{intersection}_节假日分方向分时段车流量_带聚类标签.csv'), index=False, encoding='gbk')
            print(pivot_table_holiday.head())
    

工作日透视表列名: ['小时', 1, 2, 3, 4, '聚类标签', '时段']
   小时   1  2   3   4  聚类标签  时段
0   0  39  9  41  89     2  低峰
1   1  26  2  20  52     2  低峰
2   2  31  4  24  50     2  低峰
3   3   6  2  13  35     2  低峰
4   4  10  1  13  27     2  低峰
节假日透视表列名: ['小时', 1, 2, 3, 4, '聚类标签', '时段']
   小时    1    2    3    4  聚类标签  时段
0   2   11    4    5   34     2  低峰
1   3    3    0    3   10     2  低峰
2   4    3    0    4   10     2  低峰
3   5   49   25   92  137     2  低峰
4   6  270  172  567  611     2  低峰
工作日透视表列名: ['小时', 1, 2, 3, 4, '聚类标签', '时段']
   小时     1     2     3     4  聚类标签  时段
0   0  6750  4946  3347  3897     0  低峰
1   1  3557  3016  1860  2200     0  低峰
2   2  1872  1710  1062  1258     0  低峰
3   3  1404  1094   843   916     0  低峰
4   4  1278  1167  1078  1199     0  低峰
节假日透视表列名: ['小时', 1, 2, 3, 4, '聚类标签', '时段']
   小时     1     2     3     4  聚类标签  时段
0   0  2647  1818  1303  1423     0  低峰
1   1  1337  1159   827   812     0  低峰
2   2   742   643   428   487     0  低峰
3   3   576   389   313   

In [10]:
# # 定义数据路径
# data_dir = '问题2支撑材料'

# # 获取所有工作日和节假日的车流量数据文件
# workday_files = glob.glob(os.path.join(data_dir, '*工作日分方向分时段车流量_带聚类标签.csv'))
# holiday_files = glob.glob(os.path.join(data_dir, '*节假日分方向分时段车流量_带聚类标签.csv'))

# # 定义函数读取CSV文件并处理编码问题
# def read_traffic_data(file_path):
#     try:
#         # 尝试不同的编码方式
#         encodings = ['utf-8', 'gbk', 'gb2312', 'gb18030']
#         for encoding in encodings:
#             try:
#                 df = pd.read_csv(file_path, encoding=encoding)
#                 # 重命名列名
#                 if '小时' in df.columns[0]:
#                     df.rename(columns={df.columns[0]: '小时'}, inplace=True)
#                 if '聚类标签' in df.columns:
#                     df.rename(columns={'聚类标签': '聚类标签'}, inplace=True)
#                 elif '聚类标签' in df.columns[-2]:
#                     df.rename(columns={df.columns[-2]: '聚类标签'}, inplace=True)
#                 if '时段' in df.columns:
#                     df.rename(columns={'时段': '时段'}, inplace=True)
#                 elif '时段' in df.columns[-1]:
#                     df.rename(columns={df.columns[-1]: '时段'}, inplace=True)
#                 return df
#             except UnicodeDecodeError:
#                 continue
#         raise Exception(f"无法读取文件: {file_path}")
#     except Exception as e:
#         print(f"读取文件 {file_path} 时出错: {e}")
#         return None

# # 读取所有数据文件
# workday_data = {}
# holiday_data = {}

# for file in workday_files:
#     intersection_name = os.path.basename(file).split('_工作日')[0]
#     df = read_traffic_data(file)
#     if df is not None:
#         workday_data[intersection_name] = df

# for file in holiday_files:
#     intersection_name = os.path.basename(file).split('_节假日')[0]
#     df = read_traffic_data(file)
#     if df is not None:
#         holiday_data[intersection_name] = df

# # 打印读取的交叉口数量
# print(f"工作日数据: {len(workday_data)} 个交叉口")
# print(f"节假日数据: {len(holiday_data)} 个交叉口")

# # 检查数据结构
# for name, df in workday_data.items():
#     print(f"交叉口: {name}, 列数: {len(df.columns)}, 行数: {len(df)}")

In [11]:
# # 定义路段信息
# # 纬中路上的交叉口，从西到东排序
# wei_intersections = [
#     '环西路-纬中路',
#     '经五路-纬中路',
#     '经四路-纬中路',
#     '经三路-纬中路',
#     '经中路-纬中路',
#     '经二路-纬中路',
#     '经一路-纬中路',
#     '纬中路-景区出入口',
#     '环东路-纬中路'
# ]

# # 经中路上的交叉口，从北到南排序
# jing_intersections = [
#     '经中路-环北路',
#     '经中路-纬一路',
#     '经中路-纬中路',
#     '经中路-环南路'
# ]

# # 定义路段距离（单位：km）
# # 纬中路总长约3.5公里，假设均匀分布
# wei_segment_distances = [0.4, 0.4, 0.4, 0.4, 0.7, 0.4, 0.4, 0.4]  # 8个路段

# # 经中路总长约1.8公里，假设均匀分布
# jing_segment_distances = [0.6, 0.6, 0.6]  # 3个路段

# # 定义交叉口方向信息
# direction_mapping = {
#     1: '东向西',
#     2: '西向东',
#     3: '南向北',
#     4: '北向南'
# }

In [12]:
# with tf.device('/GPU:0'):
#     # 实现格林希尔兹模型
#     class GreenshieldsModel:
#         def __init__(self, v_f=40, k_j=120):
#             """
#             初始化格林希尔兹模型
            
#             参数:
#             v_f: 自由流速度 (km/h)
#             k_j: 阻塞密度 (辆/km)
#             """
#             self.v_f = v_f  # 自由流速度
#             self.k_j = k_j  # 阻塞密度
        
#         def speed(self, k):
#             """
#             计算给定密度下的速度
            
#             参数:
#             k: 车流密度 (辆/km)
            
#             返回:
#             v: 车流速度 (km/h)
#             """
#             # 确保密度不超过阻塞密度
#             k = np.minimum(k, self.k_j)
            
#             # 格林希尔兹模型: v = v_f * (1 - k/k_j)
#             v = self.v_f * (1 - k / self.k_j)
            
#             # 确保速度非负
#             v = np.maximum(v, 0)
            
#             return v
        
#         def flow(self, k):
#             """
#             计算给定密度下的流量
            
#             参数:
#             k: 车流密度 (辆/km)
            
#             返回:
#             q: 车流量 (辆/h)
#             """
#             # q = k * v
#             q = k * self.speed(k)
#             return q
        
#         def density_from_flow(self, q):
#             """
#             根据流量计算密度
            
#             参数:
#             q: 车流量 (辆/h)
            
#             返回:
#             k: 车流密度 (辆/km)
#             """
#             # 确保流量不超过最大流量
#             q_max = self.flow(self.k_j / 2)  # 最大流量发生在 k = k_j/2
#             q = np.minimum(q, q_max)
            
#             # 从二次方程 q = k * v_f * (1 - k/k_j) 求解 k
#             # 展开得 q = v_f * k - v_f * k^2 / k_j
#             # 即 v_f * k^2 / k_j - v_f * k + q = 0
#             # 使用求根公式
#             a = self.v_f / self.k_j
#             b = -self.v_f
#             c = q
            
#             # 二次方程有两个根，我们取较小的那个（对应于非拥堵状态）
#             discriminant = b**2 - 4*a*c
#             discriminant = np.maximum(discriminant, 0)  # 确保判别式非负
            
#             k1 = (-b - np.sqrt(discriminant)) / (2*a)
#             k2 = (-b + np.sqrt(discriminant)) / (2*a)
            
#             # 选择在 [0, k_j] 范围内的有效解
#             k = np.where((k1 >= 0) & (k1 <= self.k_j), k1, k2)
            
#             return k

In [13]:
# with tf.device('/GPU:0'):
#     # 计算路段车流特性
#     def calculate_segment_characteristics(data, intersections, segment_distances, model, time_period='平峰'):
#         """
#         计算路段的车流特性（密度、速度、流量）
        
#         参数:
#         data: 字典，包含各交叉口的车流量数据
#         intersections: 列表，交叉口名称，按顺序排列
#         segment_distances: 列表，各路段距离
#         model: GreenshieldsModel实例
#         time_period: 字符串，时段类型（'低峰', '平峰', '高峰'）
        
#         返回:
#         segment_chars: 字典，包含各路段的车流特性
#         """
#         segment_chars = {}
        
#         # 遍历所有路段
#         for i in range(len(intersections) - 1):
#             start_intersection = intersections[i]
#             end_intersection = intersections[i+1]
#             segment_name = f"{start_intersection}到{end_intersection}"
#             segment_distance = segment_distances[i]
            
#             # 获取起始交叉口的出口流量和终点交叉口的入口流量
#             if start_intersection in data and end_intersection in data:
#                 start_df = data[start_intersection]
#                 end_df = data[end_intersection]
                
#                 # 筛选指定时段的数据
#                 start_df_filtered = start_df[start_df['时段'] == time_period]
#                 end_df_filtered = end_df[end_df['时段'] == time_period]
                
#                 # 计算平均小时流量
#                 # 假设方向2是(由西向东)，方向1是(由东向西)
#                 outflow = start_df_filtered[2].mean() if 2 in start_df_filtered.columns else 0
#                 inflow = end_df_filtered[1].mean() if 1 in end_df_filtered.columns else 0

#                 # 方向3是(由南向北)，方向4是(由北向南)
#                 outflow3 = start_df_filtered[3].mean() if 3 in start_df_filtered.columns else 0
#                 inflow4 = end_df_filtered[4].mean() if 4 in end_df_filtered.columns else 0
#                 # 计算路段平均流量（取出入口流量的平均值）
#                 avg_flow = (outflow + inflow + outflow3 + inflow4) / 4
                
#                 # 使用模型计算密度
#                 density = model.density_from_flow(avg_flow)
                
#                 # 计算速度
#                 speed = model.speed(density)
                
#                 # 存储结果
#                 segment_chars[segment_name] = {
#                     'distance': segment_distance,
#                     'flow': avg_flow,
#                     'density': density,
#                     'speed': speed
#                 }
        
#         return segment_chars

In [14]:
# with tf.
#     # 信号灯优化模型
#     class SignalOptimizer:
#         def __init__(self, intersections, flow_data, saturation_flow=1800, 
#                     min_green=10, max_green=60, min_cycle=60, max_cycle=120):
#             """
#             初始化信号灯优化器
            
#             参数:
#             intersections: 列表，交叉口名称
#             flow_data: 字典，包含各交叉口的车流量数据
#             saturation_flow: 饱和流率 (辆/小时)
#             min_green: 最小绿灯时间 (秒)
#             max_green: 最大绿灯时间 (秒)
#             min_cycle: 最小周期长度 (秒)
#             max_cycle: 最大周期长度 (秒)
#             """
#             self.intersections = intersections
#             self.flow_data = flow_data
#             self.saturation_flow = saturation_flow
#             self.min_green = min_green
#             self.max_green = max_green
#             self.min_cycle = min_cycle
#             self.max_cycle = max_cycle
            
#             # 相位定义：假设每个交叉口有两个相位（东西向和南北向）
#             self.phases = ['东西向', '南北向']
            
#             # 黄灯时间和全红时间
#             self.yellow_time = 3  # 秒
#             self.all_red_time = 2  # 秒
        
#         def optimize_single_intersection(self, intersection_name, time_period='平峰'):
#             """
#             优化单个交叉口的信号配时
            
#             参数:
#             intersection_name: 交叉口名称
#             time_period: 时段类型
            
#             返回:
#             optimal_timing: 字典，包含最优信号配时
#             """
#             if intersection_name not in self.flow_data:
#                 print(f"交叉口 {intersection_name} 数据不存在")
#                 return None
            
#             df = self.flow_data[intersection_name]
#             df_filtered = df[df['时段'] == time_period]
            
#             # 计算各方向的平均流量
#             direction_flows = {}
#             for col in df_filtered.columns:
#                 if isinstance(col, int) or (isinstance(col, str) and col.isdigit()):
#                     direction_flows[int(col)] = df_filtered[col].mean()
            
#             # 将方向流量分配到相位
#             # 假设方向1和2是东西向，方向3和4是南北向
#             phase_flows = {
#                 '东西向': sum([direction_flows.get(i, 0) for i in [1, 2]]),
#                 '南北向': sum([direction_flows.get(i, 0) for i in [3, 4]])
#             }
            
#             # 计算各相位的流量比
#             total_flow = sum(phase_flows.values())
#             flow_ratios = {phase: flow / total_flow for phase, flow in phase_flows.items()}
            
#             # 计算各相位的绿灯时间比例
#             green_ratios = flow_ratios.copy()
            
#             # 计算失去时间（黄灯+全红）
#             lost_time_per_phase = self.yellow_time + self.all_red_time
#             total_lost_time = lost_time_per_phase * len(self.phases)
            
#             # 计算最优周期长度（Webster公式）
#             # C = (1.5 * L + 5) / (1 - Y)，其中L是总损失时间，Y是关键相位的流量比之和
#             Y = sum(green_ratios.values())
#             optimal_cycle = (1.5 * total_lost_time + 5) / (1 - min(Y, 0.95))
#             optimal_cycle = max(min(optimal_cycle, self.max_cycle), self.min_cycle)
            
#             # 计算各相位的绿灯时间
#             effective_green_time = optimal_cycle - total_lost_time
#             green_times = {phase: max(self.min_green, green_ratio * effective_green_time) 
#                         for phase, green_ratio in green_ratios.items()}
            
#             # 调整绿灯时间，确保总和不超过有效绿灯时间
#             total_green = sum(green_times.values())
#             if total_green > effective_green_time:
#                 scale_factor = effective_green_time / total_green
#                 green_times = {phase: max(self.min_green, green_time * scale_factor) 
#                             for phase, green_time in green_times.items()}
            
#             # 计算实际周期长度
#             actual_cycle = sum(green_times.values()) + total_lost_time
            
#             # 计算各相位的绿信比
#             green_ratios_actual = {phase: green_time / actual_cycle 
#                                 for phase, green_time in green_times.items()}
            
#             # 计算各相位的通行能力和饱和度
#             capacities = {phase: self.saturation_flow * green_ratio 
#                         for phase, green_ratio in green_ratios_actual.items()}
#             saturation_degrees = {phase: phase_flows[phase] / capacity 
#                                 for phase, capacity in capacities.items()}
            
#             # 返回优化结果
#             return {
#                 'cycle_length': actual_cycle,
#                 'green_times': green_times,
#                 'phase_flows': phase_flows,
#                 'capacities': capacities,
#                 'saturation_degrees': saturation_degrees
#             }
        
#         def optimize_corridor(self, intersections, segment_distances, design_speed=30, time_period='平峰'):
#             """
#             优化交通走廊的信号配时，实现绿波控制
            
#             参数:
#             intersections: 列表，走廊上的交叉口名称，按顺序排列
#             segment_distances: 列表，相邻交叉口之间的距离
#             design_speed: 设计速度 (km/h)
#             time_period: 时段类型
            
#             返回:
#             corridor_timing: 字典，包含走廊上各交叉口的信号配时
#             """
#             # 首先优化各交叉口的信号配时
#             intersection_timings = {}
#             for intersection in intersections:
#                 timing = self.optimize_single_intersection(intersection, time_period)
#                 if timing:
#                     intersection_timings[intersection] = timing
            
#             # 如果没有足够的交叉口数据，无法进行绿波协调
#             if len(intersection_timings) < 2:
#                 return intersection_timings
            
#             # 计算公共周期（取所有交叉口周期的平均值）
#             cycles = [timing['cycle_length'] for timing in intersection_timings.values()]
#             common_cycle = sum(cycles) / len(cycles)
#             common_cycle = round(common_cycle)  # 取整
            
#             # 调整各交叉口的周期长度为公共周期
#             for intersection, timing in intersection_timings.items():
#                 # 按比例调整绿灯时间
#                 scale_factor = common_cycle / timing['cycle_length']
#                 for phase in timing['green_times']:
#                     timing['green_times'][phase] *= scale_factor
                
#                 # 更新周期长度
#                 timing['cycle_length'] = common_cycle
            
#             # 计算绿波带宽和偏移量
#             # 设计速度转换为 m/s
#             design_speed_ms = design_speed * 1000 / 3600
            
#             # 计算相邻交叉口之间的行程时间（秒）
#             travel_times = [distance * 1000 / design_speed_ms for distance in segment_distances]
            
#             # 设置第一个交叉口的偏移量为0
#             offsets = [0]
            
#             # 计算后续交叉口的偏移量
#             for i in range(len(travel_times)):
#                 # 下一个交叉口的偏移量 = 当前交叉口偏移量 + 行程时间（模公共周期）
#                 next_offset = (offsets[i] + travel_times[i]) % common_cycle
#                 offsets.append(next_offset)
            
#             # 将偏移量添加到信号配时中
#             for i, intersection in enumerate(intersections):
#                 if intersection in intersection_timings:
#                     intersection_timings[intersection]['offset'] = offsets[i]
            
#             return intersection_timings

In [15]:
# # 敏感性分析

# # 分析不同设计速度对绿波带宽的影响
# def analyze_design_speed_sensitivity(corridor_timing, segment_distances, speed_range):
#     bandwidths = []
#     for speed in speed_range:
#         bandwidth = calculate_green_wave_bandwidth(corridor_timing, segment_distances, speed)
#         bandwidths.append(bandwidth)
#     return bandwidths

# # 分析不同周期长度对平均延误的影响
# def analyze_cycle_sensitivity(intersections, flow_data, time_period='平峰', cycle_range=range(60, 121, 10)):
#     delays = []
#     for cycle in cycle_range:
#         # 创建固定周期的信号优化器
#         optimizer = SignalOptimizer(intersections, flow_data, min_cycle=cycle, max_cycle=cycle)
        
#         # 优化各交叉口
#         intersection_timings = {}
#         for intersection in intersections:
#             timing = optimizer.optimize_single_intersection(intersection, time_period)
#             if timing:
#                 intersection_timings[intersection] = timing
        
#         # 计算平均延误
#         total_delay = 0
#         total_flow = 0
#         for intersection, timing in intersection_timings.items():
#             for phase, flow in timing['phase_flows'].items():
#                 green_time = timing['green_times'][phase]
#                 green_ratio = green_time / cycle
#                 saturation_degree = timing['saturation_degrees'][phase]
#                 X = min(saturation_degree, 0.95)
                
#                 delay = 0.5 * cycle * (1 - green_ratio)**2 / (1 - X * green_ratio)
#                 total_delay += delay * flow
#                 total_flow += flow
        
#         avg_delay = total_delay / total_flow if total_flow > 0 else 0
#         delays.append(avg_delay)
    
#     return delays

# # 执行敏感性分析
# speed_range = range(20, 51, 5)  # 20-50 km/h，步长5
# wei_bandwidths_workday = analyze_design_speed_sensitivity(wei_corridor_timing_workday, wei_segment_distances, speed_range)
# jing_bandwidths_workday = analyze_design_speed_sensitivity(jing_corridor_timing_workday, jing_segment_distances, speed_range)

# cycle_range = range(60, 121, 10)  # 60-120秒，步长10
# wei_delays_workday = analyze_cycle_sensitivity(wei_intersections, workday_data, '平峰', cycle_range)
# jing_delays_workday = analyze_cycle_sensitivity(jing_intersections, workday_data, '平峰', cycle_range)

# # 可视化敏感性分析结果
# plt.figure(figsize=(12, 5))

# # 设计速度对绿波带宽的影响
# plt.subplot(1, 2, 1)
# plt.plot(speed_range, wei_bandwidths_workday, 'o-', label='纬中路')
# plt.plot(speed_range, jing_bandwidths_workday, 's-', label='经中路')
# plt.title('设计速度对绿波带宽的影响')
# plt.xlabel('设计速度 (km/h)')
# plt.ylabel('绿波带宽 (秒)')
# plt.grid(True, linestyle='--', alpha=0.7)
# plt.legend()

# # 周期长度对平均延误的影响
# plt.subplot(1, 2, 2)
# plt.plot(cycle_range, wei_delays_workday, 'o-', label='纬中路')
# plt.plot(cycle_range, jing_delays_workday, 's-', label='经中路')
# plt.title('周期长度对平均延误的影响')
# plt.xlabel('周期长度 (秒)')
# plt.ylabel('平均延误 (秒/车)')
# plt.grid(True, linestyle='--', alpha=0.7)
# plt.legend()

# plt.tight_layout()
# plt.show()

In [16]:
with tf.device('/GPU:0'):
    import pygad
    import numpy as np

    class GreenshieldsModel:
        def __init__(self, free_flow_speed=40.0, jam_density=120.0):
            """
            初始化格林希尔兹模型。
            
            :param free_flow_speed: 自由流速度 (km/h)，默认40 km/h。
            :param jam_density: 阻塞密度 (辆/km)，默认120辆/km。
            """
            self.vf = free_flow_speed
            self.kj = jam_density

        def calculate_speed(self, density):
            """
            使用格林希尔兹模型计算速度。
            
            :param density: 车流密度 (辆/km)。
            :return: 速度 (km/h)。
            """
            if density > self.kj:
                return 0.0
            return self.vf * (1 - density / self.kj)

    class Intersection:
        def __init__(self, flows, saturation_flows, phase_map, lost_time=10):
            """
            初始化交叉口。
            
            :param flows: 12个每小时车流量 (辆/小时)，顺序为[东左, 东右, 东直, 西左, 西右, 西直, 南左, 南右, 南直, 北左, 北右, 北直]。
            :param saturation_flows: 12个饱和流量 (辆/小时)，同上顺序。
            :param phase_map: 12个整数，映射每个运动到相位 (0-3)。
            :param lost_time: 每周期失时 (秒)，默认10秒。
            """
            self.flows = np.array(flows)
            self.saturation_flows = np.array(saturation_flows)
            self.phase_map = np.array(phase_map)
            self.num_phases = len(set(phase_map))
            self.lost_time = lost_time

        def calculate_total_delay(self, green_times):
            """
            计算交叉口总延迟。
            
            :param green_times: 每个相位的绿灯时间 (秒)。
            :return: 总延迟 (车辆-秒/小时)。
            """
            if len(green_times) != self.num_phases:
                return float('inf')
            green_times = np.array(green_times)
            if np.any(green_times < 10):
                return float('inf')
            sum_g = np.sum(green_times)
            if sum_g < 50 or sum_g > 170:
                return float('inf')
            C = sum_g + self.lost_time
            total_delay = 0
            for m in range(len(self.flows)):
                j = self.phase_map[m]
                g_j = green_times[j]
                λ_m = g_j / C
                q_m = self.flows[m]
                s_m = self.saturation_flows[m]
                Y_m = q_m / s_m
                if λ_m * Y_m >= 1:
                    d_m = float('inf')
                else:
                    d_m = (C / 2) * ((1 - λ_m)**2) / (1 - λ_m * Y_m)
                if np.isinf(d_m):
                    total_delay += 1e6
                else:
                    total_delay += q_m * d_m
            return total_delay

    def optimize_signals(intersection, num_generations=100, sol_per_pop=50):
        """
        使用遗传算法优化信号灯配置。
        
        :param intersection: Intersection对象。
        :param num_generations: 遗传算法代数，默认100。
        :param sol_per_pop: 种群大小，默认50。
        :return: 优化的绿灯时间和最小总延迟。
        """
        num_parents = 20
        num_genes = intersection.num_phases
        init_range_low = 10
        init_range_high = 42.5

        def fitness_func(solution, sol_idx):
            delay = intersection.calculate_total_delay(solution)
            return -delay if not np.isinf(delay) else -1e6

        ga_instance = pygad.GA(
            num_generations=num_generations,
            num_parents=num_parents,
            fitness_func=fitness_func,
            sol_per_pop=sol_per_pop,
            num_genes=num_genes,
            gene_type=float,
            init_range_low=init_range_low,
            init_range_high=init_range_high,
            parent_selection_type="sss",
            crossover_type="single_point",
            mutation_type="random",
            mutation_percent_genes=10
        )

        ga_instance.run()
        solution, solution_fitness, solution_idx = ga_instance.best_solution()
        min_delay = -solution_fitness
        return solution, min_delay


ModuleNotFoundError: No module named 'pygad'

In [ ]:
# # 示例使用
# with tf.device('/GPU:0'):
#     # 导入数据

#     # 工作日高峰时段车流量（每小时平均，假设总计72小时高峰）
#     flows_per_hour = [

#     ]
#     saturation_flows = [1200, 1800, 1800] * 4  # 左转1200，右转/直行1800
#     phase_map = [3, 1, 1, 3, 1, 1, 2, 0, 0, 2, 0, 0]  # 相位映射

#     # 创建交叉口对象
#     intersection = Intersection(flows_per_hour, saturation_flows, phase_map)

#     # 优化信号灯
#     optimal_greens, min_delay = optimize_signals(intersection)

#     print("优化的绿灯时间:", optimal_greens)
#     print("最小总延迟:", min_delay)

#     # 示例格林希尔兹模型使用
#     greenshields = GreenshieldsModel()
#     density = 60  # 示例密度
#     speed = greenshields.calculate_speed(density)
#     print(f"密度 {density} 辆/km 时的速度: {speed} km/h")